In [5]:
import csv
import math
import numpy as np

BASE_TWO = 2
Outlook = ["Sunny", "Overcast", "Rain"]
Temperature = ["Hot", "Mild", "Cool"]
Humidity = ["High", "Normal"]
Wind = ["Weak", "Strong"]
PlayTennis = ["Yes", "No"]
conditions = {"Outlook": Outlook, "Temperature": Temperature, "Humidity": Humidity, "Wind": Wind, "PlayTennis": PlayTennis}


def calculate_partial_entropy(probability):
    if probability:
        log_term = math.log(probability, BASE_TWO)
    else:   # handle pure probabilities where p=0, log cannot take non-positive values
        log_term = 0
    entropy = - (probability * log_term)
    return entropy


def calculate_information_gain(parent_entropy, weights, condition_classes_probabilities):
    for i in range(len(weights)):
        parent_entropy -= weights[i] * (calculate_partial_entropy(condition_classes_probabilities[i*2]) + calculate_partial_entropy(condition_classes_probabilities[i*2 + 1]))
    return parent_entropy

def read_csv(label, dataTable, csvFile):
    with open(csvFile, mode='r')as file:
        csvFile = csv.reader(file)
        for lines in csvFile:
            dataTable.append(lines)     # Append rows of csv file to dataTable
    for i in range(len(dataTable[0])):
        label.append(dataTable[0][i])   # Append Column title to label array
    dataTable = dataTable[1:]       # Remove first row of dataTable (repeat of label array)
    return label, dataTable


def information_gain():
    csv_name = 'PlayTennis - Sheet1.csv'
    labels = []
    labels_informationGain = []
    dataTable = []
    labels, dataTable = read_csv(labels, dataTable, csv_name)

    # Calculate Parent Entropy (Same for all conditions)
    parent_probabilities = []
    for result in PlayTennis:
        tempSum = 0
        for row in dataTable:
            tempSum += row.count(result)
        parent_probabilities.append(tempSum/len(dataTable))
    parent_entropy = 0
    for i in range(len(parent_probabilities)):
        parent_entropy += calculate_partial_entropy(parent_probabilities[i])
    # print(f"parent entropy: {parent_entropy}")

    # Calculate the condition entropies
    for label in labels:
        if label == "PlayTennis":
            continue
        condition_classes_probabilities = []
        weights = []
        for conditionClass in conditions[label]:
            condition_classes_sum = 0
            playTennis_sum = 0
            no_playTennis_sum = 0
            for row in dataTable:
                condition_classes_sum += row.count(conditionClass)      # Sum of occurrence of a Condition Class ex. Class: Sunny appears 5 times in dataTable for Condition: Outlook
                if row.count(conditionClass):                           # Find the "PlayTennis" and not "PlayTennis" sums for this class
                    playTennis_sum += row.count(PlayTennis[0])
                    no_playTennis_sum += row.count(PlayTennis[1])
            # print(f"{conditionClass} has {playTennis_sum} yes, and {no_playTennis_sum} no and a TOTAL of {condition_classes_sum}")
            condition_classes_probabilities.append(playTennis_sum/condition_classes_sum)        # Add probability of playing tennis for this class
            condition_classes_probabilities.append(no_playTennis_sum / condition_classes_sum)   # Add probability of not playing tennis for this class
            weights.append(condition_classes_sum/len(dataTable))                                # Add weight of this class for Gain calculation
            # print(condition_classes_sum)
            # print(labels)
        condition_information_gain = calculate_information_gain(parent_entropy, weights, condition_classes_probabilities)   # Calculate the Information Gain
        labels_informationGain.append(condition_information_gain)

    root_of_decisionTree = labels[np.argmax(labels_informationGain)]
    print(f"The attribute which should be at the root of the decision tree is: {root_of_decisionTree}, with the highest information gain of: {labels_informationGain[np.argmax(labels_informationGain)]}")
    print("\nThe calculated Information Gains for the conditions are as follows:")
    print("CONDITION      |      INFORMATION GAIN")
    for i in range(len(labels_informationGain)):
        print(f"{labels[i]}      |      {labels_informationGain[i]}")


information_gain()

FileNotFoundError: [Errno 2] No such file or directory: 'PlayTennis - Sheet1.csv'